Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [199]:
# Use a subset of the data with one family dwellings

# Condition 
condition = (
    (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
    (df['SALE_PRICE'] > 100000) &
    (df['SALE_PRICE'] < 2000000)
)

# One family dwelling subset
ofd = df[condition].copy()

In [200]:
# Convert date column to datetime
ofd['SALE_DATE'] = pd.to_datetime(ofd['SALE_DATE'])



In [201]:
# Convert year to int64
ofd['YEAR_BUILT'] = ofd['YEAR_BUILT'].astype(int)

In [202]:
ofd['LAND_SQUARE_FEET'] = pd.to_numeric(ofd['LAND_SQUARE_FEET'].str.replace(',', ''))

In [203]:
# Convert zip codes to strings
ofd['ZIP_CODE'] = (
    ofd['ZIP_CODE']
    .astype(int)
    .astype(str)
)

In [204]:
# Drop some problematic columns
ofd = ofd.drop(['EASE-MENT', 'APARTMENT_NUMBER',
                    'TAX_CLASS_AT_TIME_OF_SALE'], axis=1)

In [205]:
# Do train/test split
train = ofd[ofd['SALE_DATE'].dt.month < 4]
test = ofd[ofd['SALE_DATE'].dt.month == 4]

In [206]:
# High cardinality categorical values to exclude
train.describe(exclude='number').T.sort_values(by='unique', ascending=False)

,count,unique,top,freq,first,last
ADDRESS,2507,2497,130-52 LEFFERTS BOULEVARD,2,NaT,NaT
ZIP_CODE,2507,122,10312,115,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
NEIGHBORHOOD,2507,7,OTHER,2360,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaT,NaT


In [288]:
# Do one-hot encoding of categorical features


# Exclude high cardinality or unhelpful
# categorical variables from features
target = 'SALE_PRICE'
high_card = ['ADDRESS', 
             'SALE_DATE', 
             'BUILDING_CLASS_CATEGORY','TAX_CLASS_AT_PRESENT',
            'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 
            'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS','BUILDING_CLASS_AT_PRESENT']
features = train.columns.drop([target] + high_card)

In [289]:
print(features)
print(len(features))

Index(['BOROUGH', 'NEIGHBORHOOD', 'BLOCK', 'LOT', 'ZIP_CODE', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET'],
      dtype='object')
8


In [290]:
# Define feature matrices and target vectors
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [291]:
# Shape before encoding
X_train.shape

(2507, 8)

In [292]:
# Perform one-hot encoding
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [293]:
# Now there are more columns in X_train and X_test
print(X_train.shape)
print(X_test.shape)

(2507, 139)
(644, 139)


In [294]:
# Find best k and use it to find best MAE with ridge regression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

highest_mae = None
alpha_used_for_highest_mae = None
k_used_for_highest_mae = None
lowest_mae = None
best_alpha = None
k_used = None

for k in range(1, (X_train.shape[1]+1)):
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_select = selector.fit_transform(X_train, y_train)
    X_test_select = selector.transform(X_test)
    
    for alpha in [0.001, 0.01, 0.1, 1.0, 1, 100.0, 1000.0]:
        model = Ridge(alpha=alpha, normalize=True)
        model.fit(X_train_select, y_train)
        y_pred = model.predict(X_test_select)
        mae = mean_absolute_error(y_test, y_pred)
               
        if k == 1:
            lowest_mae = mae
            highest_mae = mae
            alpha_used_for_highest_mae = alpha
            k_used_for_highest_mae = k
        else:
            if mae < lowest_mae:
                lowest_mae = mae
                best_alpha = alpha
                k_used = k
            if mae > highest_mae:
                highest_mae = mae
                alpha_used_for_highest_mae = alpha
                k_used_for_highest_mae = k
#         print("K: ", k)
#         print(mae)
#         print(alpha)
    
               
print("Lowest Ridge Regression Error:")
print(f"\tMAE: ${lowest_mae:,.2f}\n\tALPHA: "\
      f"{best_alpha}\n\tK: {k} of {X_train.shape[1]}")
print("Highest Ridge Regression Error:")
print(f"\tMAE: ${highest_mae:,.2f}\n\tALPHA: "\
      f"{alpha_used_for_highest_mae}\n\tK: {k_used_for_highest_mae} of {X_train.shape[1]}")

Lowest Ridge Regression Error:
	MAE: $135,898.62
	ALPHA: 0.1
	K: 139 of 139
Highest Ridge Regression Error:
	MAE: $211,517.35
	ALPHA: 1000.0
	K: 1 of 139
